In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_nyc = pd.read_csv('data/data_snapshot_for_gdv.csv')
df_nyc.head()

In [ ]:
last_to_help = df_nyc.groupby('neighborhood').agg({'median_resolution_time_hours': 'median'}).sort_values(by='median_resolution_time_hours', ascending=False)
last_to_help

In [ ]:
last_to_solve = df_nyc.groupby('complaint_type').agg({'median_resolution_time_hours': 'median'}).sort_values(by='median_resolution_time_hours', ascending=False)
last_to_solve

In [ ]:
top_city_pains = df_nyc.groupby('complaint_type').agg({'complaint_count': 'sum', 'median_resolution_time_hours': 'median'}).sort_values(by='complaint_count', ascending=False).head(10)
top_city_pains